##**1. Importing the Required Libraries**

In [ ]:
import os

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

# librosa is a Python library for analyzing audio and music.
# It can be used to extract the data from the audio files we will see it later
import librosa
import torch
import librosa.display
import warnings
warnings.filterwarnings("ignore")
# to play the audio files
from IPython.display import Audio
plt.style.use('seaborn-white')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **2. Feature Extraction using wavLM**

In [ ]:
import torchaudio


In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 66.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 85.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 86.2 MB/s eta 0:00:00


In [ ]:
from transformers import AutoProcessor, WavLMModel, Wav2Vec2Processor
processor = Wav2Vec2Processor.from_pretrained("patrickvonplaten/wavlm-libri-clean-100h-base-plus")
wavlm = WavLMModel.from_pretrained("patrickvonplaten/wavlm-libri-clean-100h-base-plus")# audio file is decoded on the fly

def extract_features(path):
    sample_rate = 16000
    array, fs = torchaudio.load(path)
    input = processor(array.squeeze(), sampling_rate= sample_rate, return_tensors="pt")
    # apply the model to the input array from wav
    with torch.no_grad():
       outputs = wavlm(**input)
    # extract last hidden state, compute average, convert to numpy
    last_hidden_states = outputs.last_hidden_state.squeeze().mean(axis=0).numpy()
    return last_hidden_states

## **2.1 Training Dataset**

## **2.1.1 Dataset loading**

In [ ]:
csv_file_path = '/content/drive/MyDrive/Audio_Data/annotations/annotations/Haryanvi_train.csv'
# Read the CSV file into a DataFrame
har_train = pd.read_csv(csv_file_path)
har_train.head()

,filename,label
0,Abuse_1.wav,No
1,Abuse_2.wav,Yes
2,Abuse_4.wav,No
3,Abuse_5.wav,Yes
4,Abuse_6.wav,Yes


In [ ]:
har_train.shape

(818, 2)

## **2.1.2 Feature Extraction**

In [ ]:
image_directory_har = '/content/drive/MyDrive/Audio_Data/Indian_Language_Audio_Dataset/SC_audio_Haryanvi'
har_train['filename']= image_directory_har + "/" + har_train['filename']
har_train.head()

,filename,label
0,/content/drive/MyDrive/Audio_Data/Indian_Langu...,No
1,/content/drive/MyDrive/Audio_Data/Indian_Langu...,Yes
2,/content/drive/MyDrive/Audio_Data/Indian_Langu...,No
3,/content/drive/MyDrive/Audio_Data/Indian_Langu...,Yes
4,/content/drive/MyDrive/Audio_Data/Indian_Langu...,Yes


In [ ]:
#save embedding in Numpy


In [ ]:
har_train_embeddings = []
for i in range(len(har_train)):
    features = extract_features(har_train['filename'][i])
    har_train_embeddings.append(features)

har_train_embeddings = np.array(har_train_embeddings)
print(har_train_embeddings.shape)

(818, 768)


In [ ]:
np.save('/content/drive/MyDrive/Audio_Data/Output/haryanvi_train_embeddings.npy',har_train_embeddings)

print("haryanvi_train_embeddings.npy file is saved")


haryanvi_train_embeddings.npy file is saved


## **2.2 Testing Dataset**

## **2.2.1 Dataset Loading**

In [ ]:
csv_file_path_test = '/content/drive/MyDrive/Audio_Data/annotations/annotations/Haryanvi_test.csv'
# Read the CSV file into a DataFrame
har_test = pd.read_csv(csv_file_path_test)
har_test.head()

,filename,label
0,Abuse_0.wav,Yes
1,Abuse_3.wav,Yes
2,Abuse_7.wav,No
3,Abuse_11.wav,No
4,Abuse_14.wav,Yes


In [ ]:
har_test['filename']= image_directory_har + "/" + har_test['filename']
har_test.head()

,filename,label
0,/content/drive/MyDrive/Audio_Data/Indian_Langu...,Yes
1,/content/drive/MyDrive/Audio_Data/Indian_Langu...,Yes
2,/content/drive/MyDrive/Audio_Data/Indian_Langu...,No
3,/content/drive/MyDrive/Audio_Data/Indian_Langu...,No
4,/content/drive/MyDrive/Audio_Data/Indian_Langu...,Yes


In [ ]:
feats_test = np.array(extract_features(har_test['filename'][0]))
feats_test.shape

(768,)

## **2.2.2 Feature Extraction**

In [ ]:
har_test_embeddings = []
for i in range(len(har_test)):
    features_test = extract_features(har_test['filename'][i])
    har_test_embeddings.append(features_test)

har_test_embeddings = np.array(har_test_embeddings)
print(har_test_embeddings.shape)

(366, 768)


In [ ]:
np.save('/content/drive/MyDrive/Audio_Data/Output/haryanvi_test_embeddings.npy',har_test_embeddings)

print("har_test_embeddings.npy file is saved")

har_test_embeddings.npy file is saved
